In [1]:
import os
import pandas as pd
import sys
import numpy as np
from pandas.core.common import flatten
import pickle
from pathlib import Path
import librosa
import json
import soundfile as sf

In [2]:
wd = os.getcwd()
DATA = os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "processed")
FIGURES = os.path.join(os.path.sep, str(Path(wd).parents[0]), "reports", "figures", "hybfu")
AUDIO_IN = os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "raw", "in_wavs")
MEERKAT_CHANNEL = json.load(open(os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "raw", "meerkat_channel.json")))
DF = os.path.join(os.path.sep, DATA, "df_focal_reduced.pkl")

In [3]:
FRANTS = os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "frants")

if (not os.path.isdir(FRANTS)):
    os.mkdir(FRANTS)
    print("Generated directory at: ",FRANTS)
else:
    print("Directory already present")

Directory already present


In [4]:
# Read file

In [5]:
df = pd.read_pickle(DF)
df.shape

(6430, 34)

In [6]:
files = os.listdir(FRANTS)

In [37]:
fileIDs = df.callID.values
#fileIDs[0:10]
#files = os.listdir(DATA+'frants/')
#fileIDs = df.callID

In [25]:
fileIDs

array(['HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:12:04.643',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:12:15.132',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:12:15.427',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:12:39.271',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:13:17.280',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:13:45.675',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:23:38.212',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:23:40.769',
       'HM_HMB_R11_AUDIO_file_4_(2017_08_23-06_44_59)_ASWMUX221163___1:23:40.997'],
      dtype=object)

In [32]:
def get_audio(wav_loc, start_s, duration_s):
    if "SOUNDFOC" in wav_loc:
        #print("soundfoc: ", wav_loc)
        data, rate = librosa.load(wav_loc, offset=start_s, duration=duration_s, sr=None, mono=False)
        wav_filename = os.path.basename(wav_loc)
        if wav_filename in MEERKAT_CHANNEL.keys():
            channel = MEERKAT_CHANNEL[wav_filename]
        else:
            channel = 0
        data = np.asfortranarray(data[channel,:])
        #data = np.asfortranarray(data[MEERKAT_CHANNEL[wav_loc],:])
    else:
        #print("no soundfoc: ", wav_loc)
        data, rate = librosa.load(wav_loc, offset=start_s, duration=duration_s, sr=None)
        #print(data)
    
    if np.issubdtype(type(data[0]), np.integer):
        data = int16_to_float32(data)
        
    return data

In [9]:
import soundfile as sf

In [38]:
ys=[]
for fileID in fileIDs:
    filename = fileID+'.wav'
    
    sr = df.loc[df['callID']==fileID,'samplerate_hz'].values[0]
    dur = df.loc[df['callID']==fileID,'duration_s'].values[0]
    offset = df.loc[df['callID']==fileID,'start_s'].values[0]
    wav_loc = df.loc[df['callID']==fileID,'original_wav'].values[0]

  # add padding
    dur = dur+0.2
    offset = offset-0.1

    y = get_audio(wav_loc, offset, dur)
    
    #librosa.write_wav(os.path.join(os.path.sep,FRANTS,filename), y, sr, norm=False)
    #y = df.loc[df['callID']==fileID,'raw_audio'].values[0]

    # Write out audio as 24bit PCM WAV
    sf.write(os.path.join(os.path.sep,FRANTS,filename), y, sr, subtype='PCM_24')

In [39]:
generated = os.listdir(FRANTS)
print(len(generated))

6428


In [46]:
generated_ids = [x[:-4] for x in generated]
all_ids = list(df.callID.values)

# in all_ids, but not generated:
list(set(all_ids)-set(generated_ids))

[]

In [47]:
list(set(generated_ids)-set(all_ids))

[]

In [48]:
len(set(all_ids))

6428

In [ ]:
# shit, callID is not unique!

In [49]:
import collections

In [50]:
print([item for item, count in collections.Counter(all_ids).items() if count>1])

['HM_LT_R09_20170903-20170908_file_2_(2017_09_03-05_44_59)_ASWMUX221110___1:14:11.469', 'HM_PET_R11_20170903-20170908_file_3_(2017_09_04-05_44_59)_ASWMUX221163___19:03.754']


In [51]:
df.loc[df.callID=='HM_LT_R09_20170903-20170908_file_2_(2017_09_03-05_44_59)_ASWMUX221110___1:14:11.469',:]

,Name,Start,Duration,Time Format,Type,Description,start_s,duration_s,stop_s,date,...,raw_audio,spectrograms,denoised_spectrograms,stretched_spectrograms,stretched_denoised_spectrograms,stretched_mfccs,mfccs,zmfccs,freq_spectrograms,ampli_spectrograms
8440,ALARM,1:14:11.469,0:00.132,decimal,Cue,NaN,4451.469,0.132,4451.601,2017-09-03,...,"[-0.0234375, -0.044921875, -0.044921875, -0.03...","[[-4.8763523, -4.3278656, -3.4910812, -3.86334...","[[15.797497, 16.119806, 19.83519, 24.304077, 2...","[[-4.873928, -4.741518, -4.5996056, -4.4492283...","[[15.797497, 16.228107, 16.25417, 16.167095, 1...","[[-130.82164, -131.24345, -131.66933, -132.133...","[[-130.83699, -132.54517, -143.4861, -161.7499...","[[3.959618, 3.7962391, 2.7497873, 1.0029402, 0...","[[-24.602575, -17.309608, -10.6869, -6.1829586...","[[0.09142631, 0.21169937, 0.45379004, 0.762170..."
8441,ALARM,1:14:11.469,0:00.132,decimal,Cue,NaN,4451.469,0.132,4451.601,2017-09-03,...,"[-0.0234375, -0.044921875, -0.044921875, -0.03...","[[-4.8763523, -4.3278656, -3.4910812, -3.86334...","[[15.797497, 16.119806, 19.83519, 24.304077, 2...","[[-4.873928, -4.741518, -4.5996056, -4.4492283...","[[15.797497, 16.228107, 16.25417, 16.167095, 1...","[[-130.82164, -131.24345, -131.66933, -132.133...","[[-130.83699, -132.54517, -143.4861, -161.7499...","[[3.959618, 3.7962391, 2.7497873, 1.0029402, 0...","[[-24.602575, -17.309608, -10.6869, -6.1829586...","[[0.09142631, 0.21169937, 0.45379004, 0.762170..."


In [52]:
df.loc[df.callID=='HM_PET_R11_20170903-20170908_file_3_(2017_09_04-05_44_59)_ASWMUX221163___19:03.754',:]

,Name,Start,Duration,Time Format,Type,Description,start_s,duration_s,stop_s,date,...,raw_audio,spectrograms,denoised_spectrograms,stretched_spectrograms,stretched_denoised_spectrograms,stretched_mfccs,mfccs,zmfccs,freq_spectrograms,ampli_spectrograms
10996,SN,19:03.754,0:00.128,decimal,Cue,NaN,1143.754,0.128,1143.882,2017-09-04,...,"[-0.001953125, -0.00390625, -0.001953125, -0.0...","[[-1.922924, -1.2751312, -0.22265244, -0.27465...","[[16.243113, 16.48788, 17.131502, 18.442152, 1...","[[-1.8889809, -1.7258148, -1.5605278, -1.39357...","[[16.243113, 16.356424, 16.460875, 16.470005, ...","[[-117.239296, -116.48058, -115.54472, -114.46...","[[-117.45398, -113.59075, -111.453995, -116.60...","[[0.47103828, 1.1792225, 1.5709221, 0.6275647,...","[[-0.027219772, 0.0, -0.1610446, -0.71172285, ...","[[0.4165816, 0.41788912, 0.41021243, 0.3850126..."
10995,Marker 78,19:03.754,0:00.128,decimal,Cue,NaN,1143.754,0.128,1143.882,2017-09-04,...,"[-0.001953125, -0.00390625, -0.001953125, -0.0...","[[-1.922924, -1.2751312, -0.22265244, -0.27465...","[[16.243113, 16.48788, 17.131502, 18.442152, 1...","[[-1.8889809, -1.7258148, -1.5605278, -1.39357...","[[16.243113, 16.356424, 16.460875, 16.470005, ...","[[-117.239296, -116.48058, -115.54472, -114.46...","[[-117.45398, -113.59075, -111.453995, -116.60...","[[0.47103828, 1.1792225, 1.5709221, 0.6275647,...","[[-0.027219772, 0.0, -0.1610446, -0.71172285, ...","[[0.4165816, 0.41788912, 0.41021243, 0.3850126..."
